In [ ]:
#| default_exp regularize.regularize_callback

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from fastai.callback.all import *
from fastcore.basics import store_attr, listify
from fasterai.core.criteria import *
from fasterai.core.granularity import *
from fasterai.core.schedule import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Union, Optional, Type

In [ ]:
#| export
class RegularizeCallback(Callback):
    def __init__(self, 
                 criteria: Union[Criteria, list[Criteria]],            # Criteria(s) to use for regularization
                 granularity: Union[str, list[str]],                   # Granularity level(s) for grouping
                 weight: float = 0.01,                                 # Regularization weight
                 layer_types: Union[Type, list[Type]] = nn.Conv2d,     # Layer types to apply regularization to
                 schedule: Optional[Schedule] = None,                  # Optional schedule for regularization weight
                 per_layer_weights: Optional[dict[str, float]] = None, # Optional per-layer weights
                 verbose: bool = False                                 # Whether to report regularization weight
    ):
        "Callback to apply regularization using criteria during training"
        store_attr()
        self.criteria = listify(criteria)
        self.granularity = listify(granularity)
        self.layer_types = listify(layer_types)
        self.per_layer_weights = per_layer_weights or {}
        self.current_weight = weight
        
    def before_batch(self) -> None:
        "Update regularization weight if scheduled"
        if self.schedule is not None:
            self.current_weight = self.schedule([self.weight], self.pct_train)[0]
        
    def after_loss(self) -> None:
        "Apply regularization after computing the main loss"
        reg = self.get_norm()
        self.learn.loss_grad += reg
        self.learn.loss = self.learn.loss_grad.clone()
            
    def get_norm(self) -> torch.Tensor:
        "Compute regularization using the specified criteria and granularities"
        total_reg = 0.0
        
        for crit in self.criteria:
            for g in self.granularity:
                layer_regs = []
                
                for m in self.learn.model.modules():
                    if any(isinstance(m, lt) for lt in self.layer_types) and hasattr(m, 'weight'):
                        try:
                            scores = crit.f(m.weight)[None].abs().sum(Granularities.get_dim(m, g))
                            layer_reg = self.current_weight * scores.sum()
                            layer_regs.append(layer_reg)
                            
                        except (KeyError, ValueError) as e:
                            # Skip unsupported granularities for this module type
                            import warnings
                            warnings.warn(f"Skipping regularization for {type(m).__name__}: {e}")
                            continue
                        except RuntimeError as e:
                            # Handle tensor operation errors
                            import warnings
                            warnings.warn(f"Runtime error in regularization for {type(m).__name__}: {e}")
                            continue
                
                if layer_regs:
                    total_reg += torch.stack(layer_regs).sum()
        
        return total_reg
    
    def after_epoch(self) -> None:
        "Report current regularization weight if verbose"
        if self.verbose:
            print(f"Current regularization weight: {self.current_weight:.6f}")

In [ ]:
show_doc(RegularizeCallback)

The `RegularizeCallback`can be used to perform $l_1$ regularization on any granularity available in the `criteria` class.